# Quantum Noise Channel Characterization

This notebook walks through the modeling and analysis of common quantum noise channels.

**What we'll cover:**
1. Building Kraus operators for depolarizing, amplitude damping, and phase damping channels
2. Applying channels to simple quantum states and visualizing the effect
3. Sweeping noise parameters and comparing fidelity degradation
4. Computing diamond distance estimates between channels

The goal is to build intuition for how different noise types affect quantum states, and to quantify these effects using standard channel metrics.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from src.channels import (
    depolarizing_channel, amplitude_damping, phase_damping,
    apply_channel, verify_kraus_completeness, channel_to_choi
)
from src.channel_metrics import (
    process_fidelity, average_gate_fidelity, diamond_distance_estimate,
    state_fidelity, channel_fidelity_sweep
)
from src.simulation import compare_channels, compute_channel_output_states
from src.plotting import (
    plot_fidelity_vs_noise, plot_channel_comparison, plot_bloch_trajectory,
    plot_density_matrix, plot_diamond_distance_comparison
)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

RESULTS_DIR = Path('../results')
RESULTS_DIR.mkdir(exist_ok=True)

print('Setup complete!')

## 1. Kraus Operators and Basic Channel Effects

Let's start by building the Kraus operators for each channel type and verifying the completeness relation $\sum_k E_k^\dagger E_k = I$. Then we'll apply each channel to the $|+\rangle$ state and a Bell state to see how they differ.

In [ ]:
# build Kraus operators at moderate noise
noise_param = 0.2

dep_kraus = depolarizing_channel(noise_param)
amp_kraus = amplitude_damping(noise_param)
phase_kraus = phase_damping(noise_param)

# verify completeness
for name, kraus in [('Depolarizing', dep_kraus),
                     ('Amplitude Damping', amp_kraus),
                     ('Phase Damping', phase_kraus)]:
    valid = verify_kraus_completeness(kraus)
    print(f"{name}: completeness = {valid}, num operators = {len(kraus)}")

# define some useful input states
# |0><0|
rho_0 = np.array([[1, 0], [0, 0]], dtype=complex)
# |1><1|
rho_1 = np.array([[0, 0], [0, 1]], dtype=complex)
# |+><+|
rho_plus = 0.5 * np.array([[1, 1], [1, 1]], dtype=complex)

print("\n--- Applying channels to |+> state ---")
for name, kraus in [('Depolarizing', dep_kraus),
                     ('Amplitude Damping', amp_kraus),
                     ('Phase Damping', phase_kraus)]:
    rho_out = apply_channel(rho_plus, kraus)
    fid = state_fidelity(rho_plus, rho_out)
    print(f"\n{name} channel on |+>:")
    print(f"  Output state:\n{np.round(rho_out, 4)}")
    print(f"  State fidelity: {fid:.4f}")

## 2. Fidelity Sweep: How Fast Does Each Channel Degrade?

Now let's sweep the noise parameter from 0 to 1 and track how fidelity degrades for each channel. We'll apply each channel to both $|+\rangle$ and $|1\rangle$ states to see which states are most sensitive to each noise type.

Key predictions:
- $|+\rangle$ should be most sensitive to **phase damping** (since it has maximal coherence)
- $|1\rangle$ should be most sensitive to **amplitude damping** (since it's the excited state)

In [ ]:
param_range = np.linspace(0, 0.99, 25)

# compare channels on |+> state
print("Sweeping noise on |+> state...")
results_plus = compare_channels(rho_plus, param_range)
plot_fidelity_vs_noise(results_plus, save_dir=str(RESULTS_DIR))

# also plot the comparison panels
plot_channel_comparison(results_plus, save_dir=str(RESULTS_DIR))

# show the combined fidelity plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = {'Depolarizing': '#FF6B6B', 'Amplitude Damping': '#4ECDC4',
          'Phase Damping': '#45B7D1'}

# |+> state
ax = axes[0]
for name, data in results_plus.items():
    ax.plot(data['params'], data['fidelities'], 'o-',
            color=colors[name], linewidth=2, markersize=4, label=name)
ax.set_xlabel('Noise Parameter')
ax.set_ylabel('State Fidelity')
ax.set_title('Channels Applied to |+> State')
ax.legend(fontsize=10)
ax.set_ylim(0.3, 1.05)
ax.grid(True, alpha=0.3)

# |1> state
print("Sweeping noise on |1> state...")
results_one = compare_channels(rho_1, param_range)
ax = axes[1]
for name, data in results_one.items():
    ax.plot(data['params'], data['fidelities'], 'o-',
            color=colors[name], linewidth=2, markersize=4, label=name)
ax.set_xlabel('Noise Parameter')
ax.set_ylabel('State Fidelity')
ax.set_title('Channels Applied to |1> State')
ax.legend(fontsize=10)
ax.set_ylim(0.3, 1.05)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'state_comparison.png', dpi=150)
plt.show()

print("Done! Notice how |+> is more sensitive to phase damping,")
print("while |1> is more sensitive to amplitude damping.")

## 3. Process Fidelity and Diamond Distance

State fidelity depends on the input state. For a state-independent measure, we use **process fidelity** (how close the channel is to the identity) and **diamond distance** (worst-case distinguishability).

Let's compute both as a function of noise strength.

In [ ]:
param_range_fine = np.linspace(0, 0.5, 20)
identity = np.eye(2, dtype=complex)

# process fidelity sweep
channels = {
    'Depolarizing': depolarizing_channel,
    'Amplitude Damping': amplitude_damping,
    'Phase Damping': phase_damping,
}

print("Computing process fidelities and diamond distances...\n")

proc_fid_results = {}
diamond_results = {}
identity_kraus = [identity]  # identity channel

for name, ch_fn in channels.items():
    sweep = channel_fidelity_sweep(ch_fn, param_range_fine, identity)
    proc_fid_results[name] = sweep

    # diamond distance from identity
    dists = []
    for p in param_range_fine:
        if p == 0.0:
            dists.append(0.0)
        else:
            kraus = ch_fn(p)
            d = diamond_distance_estimate(identity_kraus, kraus)
            dists.append(d)
    diamond_results[name] = np.array(dists)

# print results table
print(f"{'Param':>6}  {'Depol F_pro':>11}  {'AmpD F_pro':>11}  {'PhaseD F_pro':>12}")
print("-" * 50)
for i, p in enumerate(param_range_fine):
    if i % 4 == 0:  # print every 4th row
        dep_f = proc_fid_results['Depolarizing']['fidelities'][i]
        amp_f = proc_fid_results['Amplitude Damping']['fidelities'][i]
        phase_f = proc_fid_results['Phase Damping']['fidelities'][i]
        print(f"{p:>6.2f}  {dep_f:>11.4f}  {amp_f:>11.4f}  {phase_f:>12.4f}")

# plot diamond distances
plot_diamond_distance_comparison(param_range_fine, diamond_results,
                                 save_dir=str(RESULTS_DIR))

# also show the process fidelity comparison
plot_channel_comparison(proc_fid_results, save_dir=str(RESULTS_DIR))

print("\nPlots saved to results/")

## 4. Bloch Sphere Trajectory

One of the most intuitive ways to see noise effects is through the Bloch sphere. Each channel contracts the Bloch sphere in different ways:
- **Depolarizing**: uniform contraction toward the center
- **Amplitude damping**: contraction toward $|0\rangle$ (north pole)
- **Phase damping**: contraction toward the z-axis (no transverse components)

Let's trace the trajectory of $|+\rangle$ under increasing noise.

In [ ]:
param_trajectory = np.linspace(0, 0.95, 30)

for name, ch_fn in channels.items():
    print(f"\nTracing Bloch trajectory for {name} channel...")
    result = compute_channel_output_states(rho_plus, ch_fn, param_trajectory)
    plot_bloch_trajectory(result['output_states'],
                          save_dir=str(RESULTS_DIR))

    # show the start and end Bloch vectors
    rho_start = result['output_states'][0]
    rho_end = result['output_states'][-1]
    rx_s = 2 * np.real(rho_start[0, 1])
    rz_s = np.real(rho_start[0, 0] - rho_start[1, 1])
    rx_e = 2 * np.real(rho_end[0, 1])
    rz_e = np.real(rho_end[0, 0] - rho_end[1, 1])
    print(f"  Start: rx={rx_s:.3f}, rz={rz_s:.3f}")
    print(f"  End:   rx={rx_e:.3f}, rz={rz_e:.3f}")

print("\nDone! Check results/ for Bloch trajectory plots.")

## Summary

**Key takeaways from this analysis:**

1. **Channel effects are state-dependent** — the same noise strength can have very different effects depending on the input state. Phase damping hits coherent superpositions hardest, while amplitude damping targets excited state population.

2. **Depolarizing is the harshest** — it degrades fidelity fastest because it mixes errors in all three Pauli directions simultaneously.

3. **Process fidelity gives a state-independent measure** — it captures the overall quality of the channel regardless of input. The average gate fidelity is the metric most commonly used in experimental benchmarking.

4. **Diamond distance provides worst-case bounds** — useful for fault-tolerance thresholds where we need guarantees that hold for all possible inputs.

5. **Bloch sphere visualization is powerful** — each channel type has a distinct geometric signature: depolarizing contracts uniformly, amplitude damping shifts toward |0>, and phase damping collapses the equator.